## AGAME Workflow. Get fetch data

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
data_directory = r'D:\Proyectos2024\Agame\Input\Flux_data\ICOSETC'
output_directory = r'D:\Proyectos2024\Agame\Output\Processed_tables'

sites_table = pd.read_csv(os.path.join(output_directory,'sites_table_filtered.csv'))

sites_table['FETCH_MAX'] = np.nan
sites_table['FETCH_70'] = np.nan
sites_table['FETCH_90'] = np.nan
sites_table['canopy_height'] = np.nan
sites_table['instrument_height'] = ''

for index, row in sites_table.iterrows():
    
    id = row['sites_ids']
    sites_folder = row['sites_folder']
    print('Process for: ', id)

    flux_file_path = os.path.join(data_directory,sites_folder,f"ICOSETC_{id}_FLUXNET_DD_L2.csv" )
    insitu = pd.read_csv(flux_file_path, index_col='TIMESTAMP', parse_dates=['TIMESTAMP'])

    fetch_file_path = os.path.join(data_directory,sites_folder,f"ICOSETC_{id}_FLUXES_L2.csv" )
    fetch = pd.read_csv(fetch_file_path, index_col='TIMESTAMP_START', parse_dates=['TIMESTAMP_START'], na_values=-9999.0)

    try:
        sites_table.loc[index, 'FETCH_MAX'] = fetch.describe()['FETCH_MAX'].loc['50%']
        sites_table.loc[index, 'FETCH_70'] = fetch.describe()['FETCH_70'].loc['50%']
        sites_table.loc[index, 'FETCH_90'] = fetch.describe()['FETCH_90'].loc['50%']

    except KeyError as e:
        print(f"KeyError: {e}. The key 'FETCH' does not exist for station {id}.")

    iheight_file_path = os.path.join(data_directory,sites_folder,f"ICOSETC_{id}_VARINFO_FLUXNET_DD_L2.csv" )
    iheight = pd.read_csv(iheight_file_path)
    filtered_df = iheight[iheight['DATAVALUE'] == 'GPP_DT_VUT_REF']
    index_gpp = filtered_df.index.values.tolist()[0] 
    iheight = iheight[index_gpp:index_gpp+5]
    iheight_value = iheight[iheight['VARIABLE'] == 'VAR_INFO_HEIGHT']
    sites_table.loc[index, 'instrument_height'] = float(iheight_value['DATAVALUE'].values.tolist()[0])

    cheight_file_path = os.path.join(data_directory,sites_folder,f"ICOSETC_{id}_ANCILLARY_L2.csv" )
    cheight = pd.read_csv(cheight_file_path)
    cheight = cheight[cheight['VARIABLE_GROUP'] == 'GRP_HEIGHTC']
    cheight = cheight[cheight['VARIABLE'] == 'HEIGHTC']
    cheight['DATAVALUE'] = cheight['DATAVALUE'].astype(float)
    sites_table.loc[index, 'canopy_height'] = cheight.describe()['DATAVALUE'].loc['75%']
    sites_table['buffer'] = 100 * (sites_table['instrument_height']-sites_table['canopy_height']*2/3) #Fetch to height ratio https://www.mdpi.com/2073-4433/10/6/299
                                                                                                     #https://nicholas.duke.edu/people/faculty/katul/Matlab_footprint.html 
    sites_table['buffer_desired'] = sites_table['buffer'] / 3
    sites_table['buffer_difference'] = sites_table['buffer_desired'] - sites_table['FETCH_90']

    sites_table.to_csv(os.path.join(output_directory,'sites_table_fetch_data.csv'), index=False) 